In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


In [ ]:

ds = tfds.load(name="imagenette", with_info=True, split=["train","validation"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(8)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(8)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# VGG 11

In [ ]:
inputs = keras.Input(shape=(224,224,3))

x = layers.Conv2D(64, 3, activation='relu')(inputs)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

VGG11 = keras.Model(inputs, outputs)
VGG11.summary()

In [4]:
#keras.utils.plot_model(VGG11, 'my_first_model.png')


In [9]:
sgd = keras.optimizers.SGD(lr=0.0003, decay=1e-6, momentum=0.9, nesterov=True)
VGG11.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'],
)


In [10]:
VGG11.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=3,
    validation_data=ds_test,
)

Epoch 1/3
   1029/Unknown - 192s 187ms/step - loss: 1.6107 - accuracy: 0.4512

KeyboardInterrupt: 

# VGG-16

In [ ]:
inputs = keras.Input(shape=(224,224,3))

x = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(4096, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

VGG11 = keras.Model(inputs, outputs)
VGG11.summary()

# Keras VGG 16

In [ ]:
VGG_keras = keras.applications.VGG16()

In [ ]:
VGG_keras.summary()